In [ ]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import lag, lead, first, last
from pyspark.sql.window import Window

#alwaysimport this for every pyspark analytics

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
conf = pyspark.SparkConf().setAppName('appName').setMaster('local')
#sc = pyspark.SparkContext(conf=conf)
sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)
categ=['ACL9']

Postgress_df = spark.read.csv("C:\\Users\\brono\\Downloads\\bigSample_prepared.csv", inferSchema = True, header = True)
Postgress_df=Postgress_df.sort('New Date_parsed')

new_col_names=['RECID','TRANSACTION_TYPE','DEBIT_ACCT_NO','DEBIT_CURRENCY','CREDIT_ACCT_NO','CREDIT_CURRENCY','CREDIT_AMOUNT','LOC_AMT_DEBITED','LOC_AMT_CREDITED','INPUTTER_1_0','Date_parsed','AUTHORISER_0_1','CO_CODE','DEPT_CODE']
Postgress_df =Postgress_df.toDF(*new_col_names)



windows_df=Postgress_df.withColumn("row_num", row_number().over(Window.partitionBy("CO_CODE").orderBy("Date_parsed")))


windows_df1=windows_df.withColumn('prev0', lag('TRANSACTION_TYPE').over(
     Window.partitionBy("CO_CODE").orderBy('Date_parsed'))
).withColumn('nex0', lead('TRANSACTION_TYPE').over(
  Window.partitionBy("CO_CODE").orderBy('Date_parsed')
)
    ).withColumn('prev1', lag('TRANSACTION_TYPE',2).over(
     Window.partitionBy("CO_CODE").orderBy('Date_parsed'))
).withColumn('nex1', lead('TRANSACTION_TYPE',2).over(
  Window.partitionBy("CO_CODE").orderBy('Date_parsed')
)
    ).withColumn('prev2', lag('TRANSACTION_TYPE',3).over(
     Window.partitionBy("CO_CODE").orderBy('Date_parsed'))
).withColumn('nex2', lead('TRANSACTION_TYPE',3).over(
  Window.partitionBy("CO_CODE").orderBy('Date_parsed')
)
    ).withColumn('prev3', lag('TRANSACTION_TYPE',4).over(
     Window.partitionBy("CO_CODE").orderBy('Date_parsed'))
).withColumn('nex3', lead('TRANSACTION_TYPE',4).over(
  Window.partitionBy("CO_CODE").orderBy('Date_parsed')
)
    )

windows_df1=windows_df1.filter(windows_df1.prev0.isin(categ)&
                               windows_df1.nex0.isin(categ)&
                               windows_df1.prev1.isin(categ)&
                               windows_df1.nex1.isin(categ)&
                               windows_df1.prev2.isin(categ)&
                               windows_df1.nex2.isin(categ)&
                               windows_df1.prev3.isin(categ)&
                               windows_df1.nex3.isin(categ)&
                               windows_df1.TRANSACTION_TYPE.isin(categ)
                               
                              )


windows_df1.show()

#windows_df1.sort("row_num").coalesce(1).write.option("header", "true").csv("C:\\Users\\brono\\Downloads\\Mwalimu\\fullpython1.csv")


